In [1]:
import pandas as pd

# import data
df_main = pd.read_csv('snow3300.csv')
pd.set_option('max_columns', None)

# Change datatypes of all time-based columns
time_columns = ['sys_updated_on', 'sys_created_on', 'closed_at', 'opened_at', 'business_duration', 'reopened_time', 'resolved_at', 'work_start', 'calendar_duration']

for item in time_columns:
    df_main[item] = pd.to_datetime(df_main[item])
    
# place all transformations here
df_main = df_main.replace(['2d5d343adb2d17009f9aec51ca9619e9'],['five9'])

# add columns here
df_main['opened_at_hour'] = df_main['opened_at'].dt.hour
df_main['opened_at_month'] = df_main['opened_at'].dt.month

In [3]:
df_main.head(2)

,u_repeat,made_sla,upon_reject,u_money_movement,sys_updated_on,child_incidents,task_effective_number,number,u_vcc,sys_updated_by,sys_created_on,u_dollar_impact,state,sys_created_by,knowledge,u_external_customer_impacted,u_ach,calendar_stc,closed_at,impact,active,priority,sys_domain_path,opened_at,business_duration,reopened_time,resolved_at,u_hub,u_avidpay_check,u_core,u_ipay,subcategory,u_avidpay_direct,short_description,work_start,close_code,u_pir_requested,u_check,business_stc,calendar_duration,notify,sys_class_name,close_notes,parent_incident,u_choice_6,sys_id,contact_type,reopened_by,incident_state,urgency,u_utility,reassignment_count,severity,approval,sys_mod_count,reopen_count,u_board,escalation,upon_approval,u_payments_delayed,category,resolved_by_username_value,opened_by_value,sys_domain_value,configuration_item_value,caller_id_value,assignment_group_value,closed_by_value,parent_incident_value,assigned_to_value,location_value,business_service_value,opened_at_hour,opened_at_month
0,False,True,cancel,False,2020-05-01 17:08:28,0,INC0038602,INC0038602,False,LWeber,2020-01-08 14:05:10,0,7,ABarrington,False,False,False,24976.0,2020-01-15 16:34:52,3,False,4,/,2020-01-08 14:05:10,1970-01-03 14:29:42,NaT,2020-01-08 21:01:26,False,False,False,False,NaN,False,"Phone displaying ""detecting network"" & no headset",2020-01-08 14:05:10,Solved (Permanently),False,False,24976.0,1970-01-08 02:29:42,1,incident,phone showing registering. if this is still ha...,NaN,1.0,00062694db1ac4105cfba895ca961915,Self Service,NaN,7,2,False,0,3,not requested,8,0,False,0,proceed,False,NaN,Shawn McCoy,7dcb6daddb582600a7895e09af961919,global,55da5d83db3057007f3dec51ca96190c,7dcb6daddb582600a7895e09af961919,292d2d9edbe79e00bbcbf2b6af961957,48864765db7377009f9aec51ca9619fe,NaN,Shawn McCoy,5d43995edbe69b0020acde82ca961994,292d2d9edbe79e00bbcbf2b6af961957,14,1
1,False,True,cancel,False,2019-07-26 18:00:01,0,INC0033055,INC0033055,False,system,2019-07-02 18:14:14,0,7,X110099,False,False,False,1466888.0,2019-07-26 18:00:01,3,False,4,/,2019-07-02 18:14:14,1970-01-07 11:28:08,2019-07-02 18:15:02,2019-07-19 17:42:22,False,False,False,False,NaN,False,Phone issues,2019-07-02 18:14:14,Solved (Work Around),False,False,559688.0,1970-01-24 23:45:47,1,incident,If you are continuing to have issues please su...,NaN,1.0,00218d5fdb9e7b00f8544023399619d0,Self Service,{'link': 'https://avidxchangedev.service-now.c...,7,2,False,0,3,not requested,9,1,False,0,proceed,False,NaN,Shawn McCoy,7eac271fdbc7e3805cfba895ca9619b6,global,55da5d83db3057007f3dec51ca96190c,7eac271fdbc7e3805cfba895ca9619b6,292d2d9edbe79e00bbcbf2b6af961957,Shawn McCoy,NaN,Shawn McCoy,5d43995edbe69b0020acde82ca961994,292d2d9edbe79e00bbcbf2b6af961957,18,7
